### steps
1. Create an sample SQL database
2. create config.ini for Mysql database connection
3. create table
4. performing SQL window on database to solve department highest salary


In [2]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object

In [4]:
def create_msql_database(name_database):
    """Function create MYSQL database"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    databases = [x[0] for x in mycursor]
    if name_database not in databases:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")
    # Check database


def create_msql_table(query_create):
    """
    Execute query to create table
    :query_create:: to create table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')
        cursor = conn.cursor()
        print('Creating Mysql table...')
        cursor.execute(query_create)
        print('Table created')
    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

def insert_msql_table(query_insert, table_values):
    """execute query to insert values to table
    :query_table:: to create scheme table
    :table_values:: to be inserted table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')

        cursor = conn.cursor()
        print('creating MYSQL table...')
        cursor.executemany(query_insert, table_values)

        conn.commit()
        print('MYSQL table created')

    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

In [ ]:
def main():
    # To create Foreign key, departmentId, in employee table, references to department table Id
    # we need to create department table first with Id as primary key
    query_create_department_table = "CREATE TABLE Department (Id INT PRIMARY KEY, Name VARCHAR(255));"
    query_create_employee_table = "CREATE TABLE Employee (Id INT PRIMARY KEY, Name VARCHAR(255), Salary INTEGER, departmentId INT FOREIGN KEY REFERENCES Department(Id));"
    query_insert_department = "INSERT INTO Logs (Id,Num) VALUES(%s,%s)"
    #change Id integer and Num values for every run, to get distinct Id
    logs = [(1, '1'),
            (2, '1'),
            (3, '1'),
            (4, '2'), 
            (5, '1'),
            (6, '2'),
            (7, '2'),
            (8, '2'),
            (9, '3'),
            (10, '3')]
    create_msql_table(query_create)
    insert_msql_table(query_insert, logs)

if __name__ == '__main__':
    main()